In [2]:
import pandas as pd
from plotnine import *
import numpy as np
import requests

/Users/luizfernandotoledo/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
# read the fines dataset. you can find the raw data here: https://drive.google.com/drive/folders/1UGMtjMEwa01bj8gX1gtzYvvQxmkHHZr6?usp=sharing 

naonulos = pd.read_csv('naonulos.csv')
naonulos.head(1)

/Users/luizfernandotoledo/.pyenv/versions/3.9.7/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3257: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.


,Nº,Tipo Infracao,Data Infração,Bioma,Estado,Município,CPF ou CNPJ,Nome Autuado,Nº A.I.,Série A.I.,Valor Multa,Nº Processo,Status Débito,Sanções Aplicadas
0,11625,Flora,06/10/2009,NaN,ACRE,ACRELANDIA,197412782-68,ACLÉCIO DANIEL ALENCAR,634051,D,2000.0,02002.000555/2009-11,Ajuizado,"70 1° Lei, 9605/98, 3° 58 Decreto, 6514/2008"


In [4]:
# arrumando colunas
naonulos.columns = naonulos.columns.str.lower().str.replace(' ', '_')
naonulos.columns = naonulos.columns.str.replace('º', '')
naonulos.columns = naonulos.columns.str.replace('í', 'i')
naonulos.columns = naonulos.columns.str.replace('ã', 'a')
naonulos.columns = naonulos.columns.str.replace('é', 'e')
naonulos.columns = naonulos.columns.str.replace('ç', 'c')
naonulos.columns = naonulos.columns.str.replace('õ', 'o')
naonulos.columns = naonulos.columns.str.replace('a.i.', 'ai')

/var/folders/dj/5c61qy0d0b9gcj0yrt1jd2300000gn/T/ipykernel_72434/413172698.py:9: FutureWarning: The default value of regex will change from True to False in a future version.


In [5]:
naonulos.head(1)

,n,tipo_infracao,dataifracao,bioma,estado,municipio,cpf_ou_cnpj,nome_autuado,n_ai,serie_ai,valor_multa,n_processo,status_debito,sancoes_aplicadas
0,11625,Flora,06/10/2009,NaN,ACRE,ACRELANDIA,197412782-68,ACLÉCIO DANIEL ALENCAR,634051,D,2000.0,02002.000555/2009-11,Ajuizado,"70 1° Lei, 9605/98, 3° 58 Decreto, 6514/2008"


In [6]:
naonulos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215021 entries, 0 to 215020
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   n                  215021 non-null  int64  
 1   tipo_infracao      215021 non-null  object 
 2   dataifracao        215021 non-null  object 
 3   bioma              193969 non-null  object 
 4   estado             215021 non-null  object 
 5   municipio          215021 non-null  object 
 6   cpf_ou_cnpj        214692 non-null  object 
 7   nome_autuado       215021 non-null  object 
 8   n_ai               215021 non-null  object 
 9   serie_ai           186872 non-null  object 
 10  valor_multa        215021 non-null  float64
 11  n_processo         206020 non-null  object 
 12  status_debito      215021 non-null  object 
 13  sancoes_aplicadas  214819 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 23.0+ MB


In [7]:
#create year column 

naonulos.data_infracao = pd.to_datetime(naonulos.dataifracao)
naonulos['ano'] = pd.DatetimeIndex(naonulos['dataifracao']).year

naonulos.head(1)

/var/folders/dj/5c61qy0d0b9gcj0yrt1jd2300000gn/T/ipykernel_72434/2697479929.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


,n,tipo_infracao,dataifracao,bioma,estado,municipio,cpf_ou_cnpj,nome_autuado,n_ai,serie_ai,valor_multa,n_processo,status_debito,sancoes_aplicadas,ano
0,11625,Flora,06/10/2009,NaN,ACRE,ACRELANDIA,197412782-68,ACLÉCIO DANIEL ALENCAR,634051,D,2000.0,02002.000555/2009-11,Ajuizado,"70 1° Lei, 9605/98, 3° 58 Decreto, 6514/2008",2009


In [8]:
# petrobras - sum of fines
naonulos[naonulos.nome_autuado.str.contains('PETROBR')].valor_multa.sum()

854099207.65

In [9]:
# paid fines
quitados = naonulos[naonulos.status_debito == 'Quitado. Baixa automática'].copy(deep=True)


In [10]:
# describing paid fines 2000-2022
quitados.valor_multa.describe()

count    4.148700e+04
mean     1.293560e+04
std      3.032472e+05
min      1.000000e+00
25%      1.200000e+03
50%      2.500000e+03
75%      7.740000e+03
max      5.000000e+07
Name: valor_multa, dtype: float64

In [12]:
# paid fines (average) per year 
quitados.groupby('ano').valor_multa.mean()

ano
2009    17258.958606
2010    10003.960585
2011    26582.659619
2012    10005.586593
2013    15512.988376
2014     9940.454687
2015    11945.826121
2016     5585.279271
2017     7258.340212
2018    10892.285800
2019    11219.099474
2020     8652.244309
2021     8956.504477
Name: valor_multa, dtype: float64

In [16]:
# expired fines
prescritos_site = naonulos[(naonulos.status_debito == 'Baixado por prescrição da pret. punit. (Lei 9873/99,art.1º)') |\
                     (naonulos.status_debito == 'Baixado por prescrição intercorrente durante anál. jurídica') |\
                     (naonulos.status_debito == 'Baixado por prescrição da ação execut. (Lei 9873/99,art.2º)')]
prescritos_site.head(1)

,n,tipo_infracao,dataifracao,bioma,estado,municipio,cpf_ou_cnpj,nome_autuado,n_ai,serie_ai,valor_multa,n_processo,status_debito,sancoes_aplicadas,ano
40,18052,Flora,14/01/2009,NaN,ACRE,CRUZEIRO DO SUL,164449062-53,JONAS VILANOVA PONCE,633041,D,1800.0,02002.000079/2009-21,Baixado por prescrição da pret. punit. (Lei 98...,"70 72 Lei, 9605/98, 3° 47 Decreto, 6514/2008",2009


In [17]:
# describe expired fines

prescritos_site.valor_multa.describe()


count    3.355000e+03
mean     1.175588e+05
std      8.859619e+05
min      1.000000e-02
25%      3.000000e+03
50%      9.300000e+03
75%      4.000000e+04
max      3.158490e+07
Name: valor_multa, dtype: float64

In [15]:
# agropecuaria santa barbara fines
naonulos[naonulos.cpf_ou_cnpj == '07.336.695/0001-26']

,n,tipo_infracao,dataifracao,bioma,estado,municipio,cpf_ou_cnpj,nome_autuado,n_ai,serie_ai,valor_multa,n_processo,status_debito,sancoes_aplicadas,ano
11555,20796,Flora,26/05/2009,NaN,PARA,ELDORADO DOS CARAJAS,07.336.695/0001-26,AGROPECUÁRIA SANTA BÁRBARA XINGUARA S/A,461120,D,31584900.0,02018.001281/2009-09,Baixado por prescrição da pret. punit. (Lei 98...,"70 72 Lei, 9605/98, 48 3º Decreto, 6514/2008",2009
11556,156,Flora,28/05/2009,Amazonia,PARA,ELDORADO DOS CARAJAS,07.336.695/0001-26,AGROPECUÁRIA SANTA BÁRBARA XINGUARA S/A,687971,D,3372.0,02018.001305/2009-11,Para homologação/prazo de defesa,"53 3º Decreto, 6514/2008, 70 72 Lei, 9605/98",2009
11557,20850,Flora,28/05/2009,NaN,PARA,ELDORADO DOS CARAJAS,07.336.695/0001-26,AGROPECUÁRIA SANTA BÁRBARA XINGUARA S/A,687970,D,29230.0,02018.001283/2009-90,Análise admis/mérito de impugnação/defesa,"70 72 Lei, 9605/98, 58 3º Decreto, 6514/2008",2009
11851,2753,Flora,26/05/2009,Amazonia,PARA,MARABA,07.336.695/0001-26,AGROPECUÁRIA SANTA BÁRBARA XINGUARA S/A,461119,D,32005000.0,02018.001306/2009-66,"Para nova homologação, devido a alterações","70 72 Lei, 9605/98, 48 3º Decreto, 6514/2008",2009
13375,21076,Flora,27/05/2009,NaN,PARA,XINGUARA,07.336.695/0001-26,AGROPECUÁRIA SANTA BÁRBARA XINGUARA S/A,460315,D,50000000.0,02018.001322/2009-59,Notificado e aguardando pagamento/recurso,"70 72 Lei, 9605/98, 48 3º Decreto, 6514/2008",2009
13376,2762,Controle ambiental,27/05/2009,NaN,PARA,XINGUARA,07.336.695/0001-26,AGROPECUÁRIA SANTA BÁRBARA XINGUARA S/A,460316,D,7000000.0,02018.001323/2009-01,Análise admis/mérito de impugnação/defesa,"70 72 Lei, 9605/98, 66 3º Decreto, 6514/2008",2009
13377,2779,Flora,28/05/2009,NaN,PARA,XINGUARA,07.336.695/0001-26,AGROPECUÁRIA SANTA BÁRBARA XINGUARA S/A,687973,D,47046650.0,02018.001295/2009-14,Análise admis/mérito de impugnação/defesa,"70 72 Lei, 9605/98, 3º 48 Decreto, 6514/2008",2009


In [18]:
# companies most benefitted from expired fines (obs: santa barbara shows up with 31 million because they haven't updated the total value in this dataset for some reason, but the correct is BRL 50 milion)
pd.set_option('display.max_colwidth', None)
mais_prescricoes = prescritos_site.groupby('nome_autuado').valor_multa.sum().reset_index(name='total').sort_values('total', ascending=False).head(10)

mais_prescricoes.to_csv('mais_prescricoes.csv', index=False)
mais_prescricoes


,nome_autuado,total
114,AGROPECUÁRIA SANTA BÁRBARA XINGUARA S/A,31584900.0
719,DENIS ROBERTO ZAMIGNANI,25295000.0
2827,TRANSNORDESTINA LOGISTICA S.A.,17269500.0
750,DNIT-DEPARTAMENTO NACIONAL DE INFRAEST DE TRANSPORTES,10275000.0
2570,ROBERTO PEDRO TONIAL,8710000.0
358,ARILTON CESAR RIEDI,8067600.0
2298,NORTE ENERGIA S/A,7000000.0
1828,LUCIANO MENDANHA DE REZENDE,6332148.0
2847,UIRAMUTÃ - ADMINISTRAÇÃO E PARTICIPAÇÃO S/C LTDA.,6300000.0
1451,JOAQUIM GUIMARAES HONORIO,6000200.0


In [19]:
# reading our FOIA records about expired fines

prescritos_foia = pd.read_excel('planilha_prescritos_lai_ate_marco.xlsx')


prescritos_foia.columns = prescritos_foia.columns.str.lower().str.replace(' ', '_')
prescritos_foia.columns = prescritos_foia.columns.str.replace('º', '')
prescritos_foia.columns = prescritos_foia.columns.str.replace('í', 'i')
prescritos_foia.columns = prescritos_foia.columns.str.replace('ã', 'a')
prescritos_foia.columns = prescritos_foia.columns.str.replace('é', 'e')
prescritos_foia.columns = prescritos_foia.columns.str.replace('ç', 'c')
prescritos_foia.columns = prescritos_foia.columns.str.replace('õ', 'o')
prescritos_foia.columns = prescritos_foia.columns.str.replace('a.i.', 'ai')



/var/folders/dj/5c61qy0d0b9gcj0yrt1jd2300000gn/T/ipykernel_72434/3174513063.py:13: FutureWarning: The default value of regex will change from True to False in a future version.


In [20]:
#merge files so we can have expiring date and the companies names
prescritos_com_data = naonulos.merge(prescritos_foia, left_on='n_processo', right_on='n_do_processo', how='inner')
prescritos_com_data.head(1)

,n,tipo_infracao,dataifracao,bioma,estado,municipio,cpf_ou_cnpj,nome_autuado,n_ai,serie_ai,...,serie,situacao_de_prescricao,status,data_daifracao,data_da_situacao,ano_y,valor_nominal,uf_sicafi,unidade_ibama_sicafi,locaiacao_do_processo
0,18052,Flora,14/01/2009,NaN,ACRE,CRUZEIRO DO SUL,164449062-53,JONAS VILANOVA PONCE,633041,D,...,D,Prescritos,Baix. Lei 9873/99(Prescr),2009-01-14 00:00:00,2017-12-20,2017,1800.0,AC,Superintendência do Ibama no Estado do Acre,NÚCLEO DE FISCALIZAÇÃO - AC


In [21]:
# average per year
prescritos_com_data.groupby('ano_y').valor_nominal.mean()

ano_y
2010      1500.000000
2011     93520.000000
2012    308823.468421
2013     38943.799877
2014     44205.643942
2015    101383.042067
2016    119585.966958
2017     40377.542197
2018     78307.416121
2019     60775.210757
2020     84170.272634
2021    356743.094369
2022    290659.209630
Name: valor_nominal, dtype: float64

# fines that will expire in 2022 - obtained through a FOIA request

In [25]:
prescricoes_22 = pd.read_excel('indicao_prescricao_atualizada.ods', engine='odf')

In [26]:
# most expensive fines that are about to expire in 2022
prescricoes_22.sort_values('Valor Nominal', ascending=False).head(50)




,Nº do Processo,Nº do Débito,Nº do Auto,Série,Nome da Pessoa,Documento (CPF/CNPJ),Situação de Prescrição,Status,Data da Infração,Data da Situação SICAFI,Data do Marco Interruptivo SEI,Data prevista para Prescrição Intercorrente,Valor Nominal,UF Sicafi,Unidade Ibama Sicafi,Localização do Processo,UF (Local do Processo),Data da Tramitação,Sistema da Tramitação,Responsável pelo Marco Interruptivo do Processo (SEI)
1998,02048.001086/2008-32,2436881,483552,D,CHAPLEAU EXPLORAÇÃO MINERAL LTDA,8106658000356,Risco de Prescrição,"AI notif. via edital, p/aleg. finais",2008-09-19 00:00:00,2016-10-05,2019-10-01 00:00:00,2022-10-01,15000000.00,DF,NUIP SEDE,SERVIÇO DE APOIO AMBIENTAL EM SANTARÉM - PA,PA,2020-12-16,SEI,MARCIO CANTUARIA NOBRE
1134,02024.002896/2008-84,1981932,505088,D,SIVAL AFONSO ESTEVAO,1153528215,Risco de Prescrição,Aguardando pagto/rec.,2008-12-12 00:00:00,2015-01-27,2019-10-24 00:00:00,2022-10-24,11744300.00,DF,NUIP SEDE,DIVISÃO DE CONTENCIOSO ADMINISTRATIVO,-,2021-03-09,SEI,GHISLENE DA SILVA ASSUNCAO
433,02012.100330/2017-47,9232470,9133712,E,LUIZ CARLOS DE SOUSA LOPES,14833344300,Risco de Prescrição,P/homolog./prazo de def.,2017-03-23 00:00:00,2017-04-18,2019-08-19 00:00:00,2022-08-19,10505000.00,MA,Superintendência do Ibama no Estado do Maranhão,NÚCLEO DE FISCALIZAÇÃO - MA,MA,2021-06-11,SEI,MARIA DA GRACA REIS RIBEIRO
1156,02024.004674/2018-78,9907671,9190318,E,ALTAIR FAGUNDES,63144433268,Risco de Prescrição,P/homolog./prazo de def.,2018-07-05 00:00:00,2018-07-09,2022-05-03 00:00:00,2022-02-13,4850000.00,DF,Instituto Brasileiro do Meio Ambiente e dos Recursos Naturais Renováveis,SUPERINTENDÊNCIA DO IBAMA NO ESTADO DE RONDÔNIA,RO,2022-06-13,SEI,CINTIA DE LIMA VERDE PORTELA
1425,02047.000104/2010-93,3183694,389254,D,E.C. DA SILVA IND E COM - ME,7492924000100,Risco de Prescrição,Aguardando pagto/rec.,2010-03-29 00:00:00,2016-04-19,2022-05-26 00:00:00,2022-02-14,4527534.00,DF,NUIP SEDE,SUPERINTENDÊNCIA DE APURAÇÃO DE INFRAÇÕES AMBIENTAIS,-,2022-05-27,SEI,RODRIGO GONÇALVES SABENÇA
45,02001.017985/2018-65,9890687,9146385,E,JOAO FERREIRA PONCE,57226377187,Risco de Prescrição,P/homolog./prazo de def.,2018-06-20 00:00:00,2018-06-20,2019-02-11 00:00:00,2022-02-11,4125000.00,DF,Coordenação Geral de Fiscalização,COORDENAÇÃO DE OPERAÇÕES DE FISCALIZAÇÃO,-,2019-07-10,SEI,LARISSA APARICIO DE SOUZA
606,02018.001299/2012-06,4679634,734374,D,ÉVERTON MARTINELLO,82299900934,Risco de Prescrição,Para nova homologação,2012-09-21 00:00:00,2018-04-17,2019-06-25 00:00:00,2022-06-25,4025000.00,PA,Superintendência do Ibama no Estado do Pará,NÚCLEO TÉCNICO SETORIAL DESCENTRALIZADO DE INSTRUÇÃO PROCESSUAL DE AUTOS DE INFRAÇÃO - PA,PA,2019-06-25,SEI,ALAN WILLAME DE SOUZA LOBO
2270,02567.000332/2009-51,2698910,500192,D,SEBASTIÃO ALVES RIBEIRO FILHO,29490480134,Risco de Prescrição,Aguardando pagto/rec.,2009-08-07 00:00:00,2016-11-07,2019-11-06 00:00:00,2022-11-06,3768000.00,DF,NUIP SEDE,NÚCLEO DE APOIO JURÍDICO - PA,PA,2020-08-27,SEI,GHISLENE DA SILVA ASSUNCAO
2272,02567.000793/2011-48,3927953,489352,D,ATAIDE SANDOVAL MOREIRA,15595390891,Risco de Prescrição,"AI notif. via edital, p/aleg. finais",2011-10-19 00:00:00,2016-01-29,2022-06-15 00:00:00,2022-01-21,3160000.00,MT,Gerência Executiva do Ibama em Barra do Garças/MT,GRUPO NACIONAL DE PREPARAÇÃO,-,2022-06-15,SEI,JOSE KARLSON CORREIA DA SILVA MELO
384,02012.001733/2018-95,9855998,9174110,E,J. M. DE QUEIROZ-ME,10297176000100,Risco de Prescrição,P/homolog./prazo de def.,2018-05-11 00:00:00,2018-05-17,2019-08-21 00:00:00,2022-08-21,3131109.00,MA,Superintendência do Ibama no Estado do Maranhão,NÚCLEO DE FISCALIZAÇÃO - MA,MA,2021-06-11,SEI,MARIA DA GRACA REIS RIBEIRO


In [24]:
# describe fines that are about to expire
prescricoes_22['Valor Nominal'].describe()/100

count        22.970000
mean       1298.618375
std        5340.002705
min           0.046600
25%          50.000000
50%         349.000000
75%        1350.000000
max      150000.000000
Name: Valor Nominal, dtype: float64

In [42]:
# change column type
prescricoes_22['Documento (CPF/CNPJ)'] = prescricoes_22['Documento (CPF/CNPJ)'].astype(object)

In [27]:
# how many companies are in this list?
len(prescricoes_22['Nome da Pessoa'].unique())

2069

In [30]:
naonulos.head(1)

,n,tipo_infracao,dataifracao,bioma,estado,municipio,cpf_ou_cnpj,nome_autuado,n_ai,serie_ai,valor_multa,n_processo,status_debito,sancoes_aplicadas,ano
0,11625,Flora,06/10/2009,NaN,ACRE,ACRELANDIA,197412782-68,ACLÉCIO DANIEL ALENCAR,634051,D,2000.0,02002.000555/2009-11,Ajuizado,"70 1° Lei, 9605/98, 3° 58 Decreto, 6514/2008",2009


In [32]:
# how many of these companies whose fines are about to expire have been sanctioned before?
pd.set_option('display.max_columns', None)

fined_before = naonulos.merge(prescricoes_22, left_on = 'nome_autuado', right_on = 'Nome da Pessoa')['nome_autuado'].value_counts()\
.reset_index(name='total_mentions')

fined_before[fined_before.total_mentions > 1].count()


index             914
total_mentions    914
dtype: int64

In [34]:
# % of companies on the list who have already been issued a fine in the past
914 / 2069

0.4417593040115998

In [35]:
fined_before

,index,total_mentions
0,PETROLEO BRASILEIRO S/A - PETROBRAS,5073
1,DNIT-DEPARTAMENTO NACIONAL DE INFRAEST DE TRANSPORTES,441
2,VIMEX-VITORIA EXPORTAÇÃO DE MADEIRAS LTDA,164
3,EQUINOR BRASIL ENERGIA LTDA.,126
4,OIAPOQUE - IND. COM. E BENEFICIAMENTO DE MADEIRAS LTDA - ME,75
...,...,...
1523,EMERSON RICARDO PERSCH,1
1524,GERALDO MAGELA PEREIRA,1
1525,ROSALANO GOMES DA SILVA,1
1526,ITAUTINGA AGRO INDUSTRIAL S/A,1


# Adjust expired fines by year values for inflation (the origi [aqui]( https://docs.google.com/spreadsheets/d/1tX6Ur9C2iz6rlWiJjvm-BVax8H6ymPlc/edit#gid=1981427643)

In [ ]:
from datetime import date
from decimal import Decimal
from calculadora_do_cidadao import Ipca

In [ ]:
ipca = Ipca()

In [ ]:
# corrigir inflacao
ipca.adjust("31/12/2021", 27318688.82)

for i in range (2000, 2022):
    print(f"ipca.adjust('31/12/{i})', valor")


In [ ]:
ano = 2000

total_prescricoes_ano = [198142.87,
11570858.52,
2890917.61,
5896310.08,
2195967.71,
4409497.48,
2644867.79,
5672704.46,
19090912.82,
62957710.74,
96108348.97,
66228246.99,
07976601.35,
82337272.52,
63675876.69,
94583449.92,
155636830.24,
53704838.17,
104062636.64,
90951815.80,
102350043.95,
137108677.28,
27318688.82]

for valor in total_prescricoes_ano:
    print(f"ipca.adjust('31/12/{ano}', {valor})")
    ano = ano + 1


In [ ]:
print(ipca.adjust('31/12/2000', 198142.87))
print(ipca.adjust('31/12/2001', 11570858.52))
print(ipca.adjust('31/12/2002', 2890917.61))
print(ipca.adjust('31/12/2003', 5896310.08))
print(ipca.adjust('31/12/2004', 2195967.71))
print(ipca.adjust('31/12/2005', 4409497.48))
print(ipca.adjust('31/12/2006', 2644867.79))
print(ipca.adjust('31/12/2007', 5672704.46))
print(ipca.adjust('31/12/2008', 19090912.82))
print(ipca.adjust('31/12/2009', 62957710.74))
print(ipca.adjust('31/12/2010', 96108348.97))
print(ipca.adjust('31/12/2011', 66228246.99))
print(ipca.adjust('31/12/2012', 7976601.35))
print(ipca.adjust('31/12/2013', 82337272.52))
print(ipca.adjust('31/12/2014', 63675876.69))
print(ipca.adjust('31/12/2015', 94583449.92))
print(ipca.adjust('31/12/2016', 155636830.24))
print(ipca.adjust('31/12/2017', 53704838.17))
print(ipca.adjust('31/12/2018', 104062636.64))
print(ipca.adjust('31/12/2019', 90951815.8))
print(ipca.adjust('31/12/2020', 102350043.95))
print(ipca.adjust('31/12/2021', 137108677.28))

In [ ]:
pd.set_option('display.float_format', lambda x: '%.9f' % x)
valores_prescritos_ano = pd.read_csv('valores_ano_inflacao_prescritos.csv')
apos_2016 = valores_prescritos_ano[valores_prescritos_ano.ano > 2016]
apos_2016.groupby('ano').valor.sum().plot()

In [ ]:
apos_2016[['ano', 'valor']].to_csv('apos_2016_corrigido.csv')

In [ ]:
# julgamentos de multas


for state in states:
    print(f'df_' + state +  ' = pd.read_csv("' + 'http://dadosabertos.ibama.gov.br/dados/SICAFI/'+ state + '/Volume/volumeJulgamentoAI.csv"'  + ', sep=";")')
   





In [ ]:
df_AC = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/AC/Volume/volumeJulgamentoAI.csv", sep=";")
df_AL = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/AL/Volume/volumeJulgamentoAI.csv", sep=";")
df_AM = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/AM/Volume/volumeJulgamentoAI.csv", sep=";")
df_AP = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/AP/Volume/volumeJulgamentoAI.csv", sep=";")
df_BA = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/BA/Volume/volumeJulgamentoAI.csv", sep=";")
df_CE = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/CE/Volume/volumeJulgamentoAI.csv", sep=";")
df_DF = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/DF/Volume/volumeJulgamentoAI.csv", sep=";")
df_ES = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/ES/Volume/volumeJulgamentoAI.csv", sep=";")
df_GO = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/GO/Volume/volumeJulgamentoAI.csv", sep=";")
df_MA = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/MA/Volume/volumeJulgamentoAI.csv", sep=";")
df_MT = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/MT/Volume/volumeJulgamentoAI.csv", sep=";")
df_MS = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/MS/Volume/volumeJulgamentoAI.csv", sep=";")
df_MG = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/MG/Volume/volumeJulgamentoAI.csv", sep=";")
df_PA = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/PA/Volume/volumeJulgamentoAI.csv", sep=";")
df_PB = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/PB/Volume/volumeJulgamentoAI.csv", sep=";")
df_PR = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/PR/Volume/volumeJulgamentoAI.csv", sep=";")
df_PE = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/PE/Volume/volumeJulgamentoAI.csv", sep=";")
df_PI = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/PI/Volume/volumeJulgamentoAI.csv", sep=";")
df_RJ = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/RJ/Volume/volumeJulgamentoAI.csv", sep=";")
df_RN = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/RN/Volume/volumeJulgamentoAI.csv", sep=";")
df_RS = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/RS/Volume/volumeJulgamentoAI.csv", sep=";")
df_RO = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/RO/Volume/volumeJulgamentoAI.csv", sep=";")
df_RR = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/RR/Volume/volumeJulgamentoAI.csv", sep=";")
df_SC = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/SC/Volume/volumeJulgamentoAI.csv", sep=";")
df_SP = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/SP/Volume/volumeJulgamentoAI.csv", sep=";")
df_SE = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/SE/Volume/volumeJulgamentoAI.csv", sep=";")
df_TO = pd.read_csv("http://dadosabertos.ibama.gov.br/dados/SICAFI/TO/Volume/volumeJulgamentoAI.csv", sep=";")

In [ ]:
# concatenate csvs for a final judgement fines dataframe

julgamentos = pd.concat([df_AC, df_AL, df_AM, df_AP, df_BA, df_CE, df_DF, df_ES, df_GO, df_MA, df_MT, df_MS, df_MG, df_PA, df_PB, df_PR, df_PE, df_PI, df_RJ, df_RN, df_RS, df_RO, df_RR, df_SC, df_SP, df_SE, df_TO])

julgamentos.to_csv('julgamentos.csv', index=False)



In [ ]:
julgamentos

In [ ]:
# arrumando colunas
julgamentos.columns = julgamentos.columns.str.lower().str.replace(' ', '_')
julgamentos.columns = julgamentos.columns.str.replace('º', '')
julgamentos.columns = julgamentos.columns.str.replace('í', 'i')
julgamentos.columns = julgamentos.columns.str.replace('ã', 'a')
julgamentos.columns = julgamentos.columns.str.replace('é', 'e')
julgamentos.columns = julgamentos.columns.str.replace('ç', 'c')
julgamentos.columns = julgamentos.columns.str.replace('õ', 'o')
julgamentos.columns = julgamentos.columns.str.replace('a.i.', 'ai')

In [ ]:
julgamentos.data_julgamento_principal

In [ ]:
julgamentos.data_julgamento_recurso = pd.to_datetime(julgamentos.data_julgamento_recurso)
julgamentos['ano'] = pd.DatetimeIndex(julgamentos['data_julgamento_recurso']).year
julgamentos.head(1)

In [ ]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

states = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']


julga_uf_ano = julgamentos.groupby(['ano', 'uf']).tipo_de_infracao.count().reset_index(name='total_no_ano')
julga_uf_ano = julga_uf_ano[(julga_uf_ano.uf.isin(states)) & (julga_uf_ano.ano < 2022) & (julga_uf_ano.ano > 2007)]
julga_uf_ano.to_csv('julgamentos_por_uf_por_ano.csv', index=False)
julga_uf_ano


In [ ]:
# julgamentos por uf por ano
(
    ggplot(julga_uf_ano,          
        aes('factor(ano)', 'total_no_ano', group=1)) 
        + geom_line()
        + theme_bw()
        + facet_wrap('uf')
        + theme(subplots_adjust={'wspace': 0.2}, figure_size=(60, 15)) # add spaceing between facets to make y-axis ticks visible
)

#         + facet_wrap('uf', scales='free_y')

In [ ]:
julgamentos.groupby('ano').tipo_de_infracao.count().reset_index(name='total_julgados')